In [142]:
from qoop.core import state, ansatz, metric
import qiskit
from qiskit import transpile
from qiskit.quantum_info import Operator, DensityMatrix, Kraus
from scipy.linalg import qr
import numpy as np

<img src = './docs/1.png' height ='800px'>

In [152]:
# Step 1: Read quantum compilation and learn how to use qoop
# https://github.com/vutuanhai237/qoop/wiki/Advance:-Custom-state-preparation

# Step 2: Implement the following function

#Create a ansatz V with n qubits
def V(num_qubits: int):
    return ansatz.stargraph (num_qubits=num_qubits)

# def Epsilon(rho, kraus_operators):
#     # K = K_noise = [\sqrt(p) I @ I, \sqrt(1-p) Z @ Z]
#     # see Eq. 1 Ref. [1]
#     return sum(K @ rho.data @ np.transpose(np.conjugate(K)) for K in kraus_operators)

# def Epsilon2(rho, unitary_matrix):
#     # K = K_noise = [\sqrt(p) I @ I, \sqrt(1-p) Z @ Z]
#     # see Eq. 1 Ref. [1]
#     return (np.transpose(np.conjugate(unitary_matrix)) @ rho.data @ unitary_matrix)

def calRho3 (rho, unitary_matrix, kraus_operators):
    rho2 = sum(K @ rho.data @ np.transpose(np.conjugate(K)) for K in kraus_operators)
    rho3 = (np.transpose(np.conjugate(unitary_matrix)) @ rho2.data @ unitary_matrix)
    return rho3

def createKraus(unitary_matrix):
    kraus_ops = []
    Q, R = qr(unitary_matrix) #(16,16) -> vector 1x16

    #print(Q.shape)
    for q in Q:
        
        q = np.expand_dims(q, 1)
        #print(q)
        #print(np.transpose(np.conjugate(q)))
        #print(q @ np.transpose(np.conjugate(q)))
        kraus_ops.append(q @ np.transpose(np.conjugate(q))) #vector 1x16

    
    return kraus_ops



In [155]:
num_qubits = 2

#Create a ansatz V with n qubits
circuit = V(num_qubits)

#Assign random parameter
num_params = circuit.num_parameters
x0 = 2 * np.pi * np.random.random(num_params)
circuit = circuit.assign_parameters(dict(zip(circuit.parameters, x0)))

print(circuit)

      ┌──────────┐   
q_0: ─┤ Ry(1.37) ├─■─
     ┌┴──────────┤ │ 
q_1: ┤ Ry(2.809) ├─■─
     └───────────┘   


In [156]:
# Get the unitary operator corresponding to the circuit
unitary_op = Operator(circuit)

# Get the unitary matrix
unitary_matrix = unitary_op.data
print (unitary_matrix.shape) #(16, 16 as num_qubits is 4)

(4, 4)


In [157]:
KrausOperators = createKraus(unitary_matrix=unitary_matrix)
print(sum(K @ np.transpose(np.conjugate(K)) for K in KrausOperators))


[[ 1.00000000e+00+0.j -1.11022302e-16+0.j -9.71445147e-17+0.j
   4.16333634e-17+0.j]
 [-1.11022302e-16+0.j  1.00000000e+00+0.j  4.16333634e-17+0.j
  -1.38777878e-17+0.j]
 [-9.71445147e-17+0.j  4.16333634e-17+0.j  1.00000000e+00+0.j
  -1.90819582e-17+0.j]
 [ 4.16333634e-17+0.j -1.38777878e-17+0.j -1.90819582e-17+0.j
   1.00000000e+00+0.j]]


In [ ]:
#rho
# rho = DensityMatrix.from_label('0' * 2)
# rho2 = Epsilon(rho=rho, kraus_operators=KrausOperators)
# print(rho2)

# rho3 = Epsilon2(rho=rho2, unitary_matrix=unitary_matrix)
# print(rho3)


In [160]:
#rho
rho = DensityMatrix.from_label('0' * num_qubits)
rho3 = calRho3(rho=rho, kraus_operators=KrausOperators, unitary_matrix=unitary_matrix)
print(rho3)


[[ 0.04769207+0.j -0.02568851+0.j -0.07530716+0.j  0.09254917+0.j]
 [-0.02568851+0.j  0.03207227+0.j  0.06239853+0.j -0.07632505+0.j]
 [-0.07530716+0.j  0.06239853+0.j  0.3707299 +0.j -0.01120718+0.j]
 [ 0.09254917+0.j -0.07632505+0.j -0.01120718+0.j  0.54950576+0.j]]


In [159]:
#cost func compare rho, rho3
def cost(rho, rho3):
    return 1-metric.compilation_trace_fidelity(rho, rho3)

print(cost(rho=rho, rho3=rho3))

#Nhận U, K trả về rho3

0.8347985494709932


In [ ]:
# Step 3: Implement the following function
# state_need_tomogaphy = ...
# rho = np.conjugate(np.transpose(state_need_tomogaphy)) @ state_need_tomogaphy   # density matrix
# rho' = Delta(rho)
# compiler = qoop.qcompilation.QuantumCompilation(U = rho', V = V())
# compiler.fit()
# compiler.plot()
# see fidelities versus iteration

<img src = './docs/2.png' height ='800px'>